In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from transformers import DebertaTokenizer
from sklearn.decomposition import TruncatedSVD
import gensim.downloader as api # works with scipy==1.10.1
import string
from transformers import RobertaModel, RobertaTokenizer
from gensim.models.keyedvectors import KeyedVectors

In [3]:
import os, sys
sys.path.append("pan-clef-2024-oppositional/")
TRAIN_DATASET_ES="Dataset-Oppositional/training/dataset_es_train.json"
TRAIN_DATASET_EN="Dataset-Oppositional/training/dataset_en_train.json"
TEST_DATASET_EN ="Dataset-Oppositional/test/dataset_en_official_test_nolabels.json"
TEST_DATASET_ES ="Dataset-Oppositional/test/dataset_en_official_test_nolabels.json"

In [4]:
from data_tools.dataset_loaders import load_dataset_classification
texts_es, labels_es, ids_es = load_dataset_classification("es", string_labels=False, positive_class='conspiracy')
texts_en, labels_en, ids_en = load_dataset_classification("en", string_labels=False, positive_class='conspiracy')


Loading official JSON es dataset
Loading official JSON en dataset


In [11]:
# Convert to a pandas dataframe: 'id': ids_es, 'text': texts_es, 'label': labels_es, 'y': labels_es
df_es = pd.DataFrame({'id': ids_es, 'text': texts_es, 'label': labels_es})
df_en = pd.DataFrame({'id': ids_en, 'text': texts_en, 'label': labels_en})

In [12]:
df_es

,id,text,label
0,2807,Fallo en Matrix 08/02/2022 Hoy el señor Joan R...,0
1,3054,Siento ya tdas las vacunas vienen contaminadas...,0
2,268,Veo que curiosamente te autoproclamados interl...,1
3,2669,[ Documental ] Vacunas : Una inyección en la o...,0
4,3205,Una sugerencia para los que se han vacunado y ...,1
...,...,...,...
3995,1056,DR . ROBERT MALONE . Co - Inventor de las tecn...,0
3996,861,Una pregunta ? La vacuna también provoca hipot...,0
3997,5248,ERIC CLAPTON : El famoso guitarrista cuenta co...,0
3998,1328,"No es médico , no es científico , no es biólog...",1


## Representation models

- Traditional form: **Latent Semantic Analysis (LSA)**. LSA is a technique used in NLP and information retrieval to
analyze the relationships between a set of documents and the
terms they contain. It's primarily employed for text classification,
clustering, and information retrieval tasks.

- Static and contextual: **fasttext-wiki-news-subwords-300** for english text. Due to this model was trained only with english text, to make better representation in spanish, we need another model. For spanish we are going to use a model similar trained with spanish text that is trained on Common Crawl and Wikipedia using fastText. This models was trained using CBOW with position-weights, in dimension 300, with character n-grams of length 5, a window of size 5 and 10 negatives. https://fasttext.cc/docs/en/crawl-vectors.html

- Contextual word embedding: **DeBERTa**. 

In [1]:
import gensim.downloader as api # works with scipy==1.10.1
info = api.info()
for item in info["models"].items():
    print(item)
    
word_vectors = api.load('fasttext-wiki-news-subwords-300')


('fasttext-wiki-news-subwords-300', {'num_records': 999999, 'file_size': 1005007116, 'base_dataset': 'Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens)', 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/fasttext-wiki-news-subwords-300/__init__.py', 'license': 'https://creativecommons.org/licenses/by-sa/3.0/', 'parameters': {'dimension': 300}, 'description': '1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).', 'read_more': ['https://fasttext.cc/docs/en/english-vectors.html', 'https://arxiv.org/abs/1712.09405', 'https://arxiv.org/abs/1607.01759'], 'checksum': 'de2bb3a20c46ce65c9c131e1ad9a77af', 'file_name': 'fasttext-wiki-news-subwords-300.gz', 'parts': 1})
('conceptnet-numberbatch-17-06-300', {'num_records': 1917247, 'file_size': 1225497562, 'base_dataset': 'ConceptNet, word2vec, GloVe, and OpenSubtitles 2016', 'reader_code': 'https://github.com/RaRe-Technologies/ge

In [14]:
nltk.download('punkt')
nltk.download('stopwords')

# Function to preprocess text
def preprocess_text(text: str,
                    stop_words=stopwords.words('english')):
    # Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove stopwords, punctuation marks, and symbols
    tokens = [w for w in tokens
              if w not in stop_words
              and w not in string.punctuation
              and w.isalnum()]
    return tokens

# Function to calculate average vector representation of a document
def document_vector(tokens: list[str],
                    word_vectors: KeyedVectors):
    # Initialize empty vector
    document_vector = np.zeros((300,))
    # Count number of words in document
    words_count = 0
    # Iterate over each word in the document
    for word in tokens:
        if word in word_vectors.key_to_index:
            # Add word vector to document vector
            document_vector += word_vectors[word]
            words_count += 1
    # Average the document vector
    if words_count != 0:
        document_vector /= words_count
    return document_vector

# Function to create LSA features
def lsa_embedding(train_texts: pd.DataFrame,
                  val_texts: pd.DataFrame,
                  n_components: int=100,
                  process_text: bool=True):
    
    if preprocess_text:
        # Preprocess text
        train_texts['text'] = train_texts['text'].apply(preprocess_text)
        val_texts['text'] = val_texts['text'].apply(preprocess_text)
    
    # Create TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer(binary=False,
                                       ngram_range=(3, 3),
                                       analyzer='char')
    train_tfidf = tfidf_vectorizer.fit_transform(train_texts['text'])
    val_tfidf = tfidf_vectorizer.transform(val_texts['text'])
    
    # Apply SVD
    svd = TruncatedSVD(n_components=n_components)
    train_lsa = svd.fit_transform(train_tfidf)
    val_lsa = svd.transform(val_tfidf)
    
    # Add LSA embeddings to the dataframes
    train_texts['lsa_embedding'] = list(train_lsa)
    val_texts['lsa_embedding'] = list(val_lsa)
    
    return train_lsa, val_lsa


def fasttext_embedding(train_texts: pd.DataFrame,
                       val_texts: pd.DataFrame):
    
    if preprocess_text:
        # Preprocess text
        train_texts['text'] = train_texts['text'].apply(preprocess_text)
        val_texts['text'] = val_texts['text'].apply(preprocess_text)
    
    word_vectors = api.load('fasttext-wiki-news-subwords-300')
    
    # TRAIN TEXTS
    # Create dictionary to store document vectors
    document_vectors = {}

    # Iterate over each document in the dataset
    for index, row in train_texts.iterrows():
        # Preprocess text
        tokens = preprocess_text(row['text'])
        # Calculate document vector
        doc_vector = document_vector(tokens, word_vectors)
        # Store document vector in dictionary
        document_vectors[index] = doc_vector
    
    # Add document vectors as a new column in the DataFrame
    train_texts['fasttext_embeeding'] = document_vectors
    
    # VAL TEXTS
    # Create dictionary to store document vectors
    document_vectors = {}

    # Iterate over each document in the dataset
    for index, row in val_texts.iterrows():
        # Preprocess text
        tokens = preprocess_text(row['text'])
        # Calculate document vector
        doc_vector = document_vector(tokens, word_vectors)
        # Store document vector in dictionary
        document_vectors[index] = doc_vector
    
    # Add document vectors as a new column in the DataFrame
    val_texts['fasttext_embeeding'] = document_vectors 
    
    return train_texts, val_texts


def roberta_embedding(train_texts: pd.DataFrame,
                      val_texts: pd.DataFrame,
                      lang: str='en'):
    # Load DeBERTa model and tokenizer
    if lang == 'es':
        model = RobertaModel.from_pretrained('bertin-project/bertin-roberta-base-spanish')
        tokenizer = RobertaTokenizer.from_pretrained('bertin-project/bertin-roberta-base-spanish')
    elif lang == 'en':
        model = RobertaModel.from_pretrained('roberta-base')
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    else:
        raise ValueError("Invalid language. Expected 'en' or 'es'.")
    
    # Function to calculate DeBERTa embeddings
    def calculate_embedding(text):
        # Tokenize text and convert to tensor
        inputs = tokenizer(text, return_tensors='pt')
        # Get model output
        outputs = model(**inputs)
        # Get embeddings from the last hidden state
        embeddings = outputs.last_hidden_state
        # Average embeddings over sequence length to get a single vector
        doc_vector = embeddings.mean(dim=1).detach().numpy()
        return doc_vector

    # Calculate DeBERTa embeddings for train_texts and val_texts
    train_texts['roberta_embedding'] = train_texts['text'] \
                                            .apply(calculate_embedding)
    val_texts['roberta_embedding'] = val_texts['text'] \
                                            .apply(calculate_embedding)

    return train_texts, val_texts

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dandr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def all_embedings(train: pd.DataFrame,
                  val: pd.DataFrame,
                  lang: str):
    
    train, val = lsa_embedding(train, val)
    train, val = fasttext_embedding(train, val)
    train, val = roberta_embedding(train, val, lang=lang)
    return train, val

# Split df_en in train and val
train_en, val_en = train_test_split(df_en, test_size=0.2, random_state=50)
train_es, val_es = train_test_split(df_es, test_size=0.2, random_state=50)

train_en, val_en = all_embedings(train_en, val_en, 'en')
train_es, val_es = all_embedings(train_es, val_es, 'es')

# Save train_en, val_en, train_es, val_es
train_en.to_csv('train_en.csv', index=False, sep=';', encoding='utf-8')
val_en.to_csv('val_en.csv', index=False, sep=';', encoding='utf-8')
train_es.to_csv('train_es.csv', index=False, sep=';', encoding='utf-8')
val_es.to_csv('val_es.csv', index=False, sep=';', encoding='utf-8')